In [1]:
cd /content/drive/My Drive/Colab Notebooks/200318_NN

/content/drive/My Drive/Colab Notebooks/200318_NN


In [0]:
import pandas as pd 

train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [0]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
X_test = test.iloc[:, 1:].values / 255

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch

In [0]:
class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        return torch.tensor(X).float(), torch.tensor(y)

class TestDataset(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X = self.X[idx]
        return torch.tensor(X).float()

In [0]:
traindataset = TrainDataset(X, y)
testdataset = TestDataset(X_test)

In [0]:
trainloader = DataLoader(traindataset, batch_size=1024, shuffle=True, num_workers=2)
testloader = DataLoader(testdataset, batch_size=1024, shuffle=False, num_workers=2)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.batch1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.batch2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.batch3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.dropout(F.relu(self.batch1(self.fc1(x))))
        x = self.dropout(F.relu(self.batch2(self.fc2(x))))
        x = self.dropout(F.relu(self.batch3(self.fc3(x))))

        x = self.fc4(x)

        return x

net = Net()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)  

cuda


In [0]:
net = net.to(device)

In [21]:
num_epochs = 100
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]       
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients: 역전파 단계 실행 전 gradient를 0으로 만든다 
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward() # autograd: 자동미분 
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    if epoch%10 == 0: 
        print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
    running_loss = 0.0

print('Finished Training')

[1,    42] loss: 0.201
[11,    42] loss: 0.158
[21,    42] loss: 0.135
[31,    42] loss: 0.112
[41,    42] loss: 0.094
[51,    42] loss: 0.084
[61,    42] loss: 0.074
[71,    42] loss: 0.067
[81,    42] loss: 0.059
[91,    42] loss: 0.052
Finished Training


In [0]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
            preds.append(pred.cpu().numpy())

In [24]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch1.csv", index=False)

kaggle score: 0.96962